In [1]:
import numpy as np
import alphas3Momentos as tablas

In [11]:
def calcular_voladizo(tramo, pos_tramo):
    M = 0
    
    for car,caract in tramo['cargas'].items():
        
        if caract['tipo']  == 'puntual':
            if pos_tramo == 0:
                M += caract['magnitud'] * (tramo['L'] - caract['distancia'])
            else:
                M += caract['magnitud'] * (caract['distancia'])
                
                
        if caract['tipo']  == 'distribuida':
            if pos_tramo == 0:
                M += caract['magnitud'] * caract['longitud'] * (tramo['L'] - 0.5 * caract['longitud'] - caract['distancia'])
            else:
                M += caract['magnitud'] * caract['longitud'] * (0.5 * caract['longitud'] + caract['distancia'])
        
        
        if caract['tipo']  == 'triangular':
            dif = caract['dist_pico'] - caract['distancia']
            
            if pos_tramo == 0:
                if dif > 0:
                    M += 0.5 * caract['magnitud'] * caract['longitud'] * (tramo['L'] - caract['distancia'] - 2/3 * caract['longitud'])
                else:
                    M += 0.5 * caract['magnitud'] * caract['longitud'] * (tramo['L'] - caract['distancia'] - 1/3 * caract['longitud'])
            else:
                if dif > 0:
                    M += 0.5 * caract['magnitud'] * caract['longitud'] * (caract['distancia'] + 2/3 * caract['longitud'])
                else:
                    M += 0.5 * caract['magnitud'] * caract['longitud'] * (caract['distancia'] + 1/3 * caract['longitud'])
                
        if caract['tipo']  == 'momento':
            
            if pos_tramo == 0:
                M += -caract['magnitud']
            else:
                M += caract['magnitud']
                
        if caract['tipo']  == 'trapesoidal':
            dif = caract['dist_mayor'] - caract['dist_menor']
            dif_mag = caract['mag_mayor'] - caract['mag_menor']
            if pos_tramo == 0:
                if dif > 0:
                    M += 0.5 * dif_mag * caract['longitud'] * (tramo['L'] - caract['dist_menor'] - 2/3 * caract['longitud'])
                    M += caract['mag_menor'] * caract['longitud'] * (tramo['L'] - 0.5 * caract['longitud'] - caract['dist_menor'])
                else: 
                    M += 0.5 * dif_mag * caract['longitud'] * (tramo['L'] - caract['dist_mayor'] - 1/3 * caract['longitud'])
                    M += caract['mag_menor'] * caract['longitud'] * (tramo['L'] - 0.5 * caract['longitud'] - caract['dist_mayor'])
            else:
                if dif > 0:
                    M += 0.5 * dif_mag * caract['longitud'] * (caract['dist_menor'] + 2/3 * caract['longitud'])
                    M += caract['mag_menor'] * caract['longitud'] * (0.5 * caract['longitud'] + caract['dist_menor'])
                else: 
                    M += 0.5 * dif_mag * caract['longitud'] * (caract['dist_mayor'] + 1/3 * caract['longitud'])
                    M += caract['mag_menor'] * caract['longitud'] * (0.5 * caract['longitud'] + caract['dist_mayor'])
        
    if pos_tramo == 0:
        return -M
    else:
        return M

In [3]:
class viga():
    def __init__(self):
        self.tramos = []        
    
    def definir_tramos(self):
        aux = True
        while aux:
            voladizo = input("Es un voladizo? ")
            if voladizo != "si":
                empotrado = input("Es un empotramiento? ")
            else:
                empotrado = "no"
                
            n_cargas = int(input('Cuantas cargas hay en el tramo? '))
            
            cargas = {}
            
            for i in range(n_cargas):
                tipo_carga = input("Escriba el tipo de la carga " + str(i+1) + " ")
                
                if tipo_carga == "puntual":
                    
                    magnitud = float(input("Escriba la magnitud de la carga "))
                    distancia = float(input("Escriba la distancia de la carga al extremo izquierdo del tramo "))
                    
                    cargas['carga ' + str(i+1)]  = {
                        'tipo': tipo_carga,
                        'magnitud': magnitud,
                        'distancia': distancia
                    }
                    
            
                if tipo_carga == "distribuida":

                    magnitud = float(input("Escriba la magnitud de la carga "))
                    distancia = float(input("Escriba la distancia de la carga al extremo izquierdo del tramo "))
                    l_carga = float(input("Escriba la longitud de la carga "))

                    cargas['carga ' + str(i+1)]  = {
                        'tipo': tipo_carga,
                        'magnitud': magnitud,
                        'distancia': distancia,
                        'longitud': l_carga
                    }
                    
                
                if tipo_carga == "triangular":
                    
                    magnitud = float(input("Escriba la magnitud de la carga "))
                    distancia = float(input("Escriba la distancia de la carga al extremo izquierdo del tramo "))
                    dist_pico = float(input("Escriba la distancia del pico al extremo izquierdo del tramo "))
                    l_carga = float(input("Escriba la longitud de la carga "))

                    cargas['carga ' + str(i+1)]  = {
                        'tipo': tipo_carga,
                        'magnitud': magnitud,
                        'distancia': distancia,
                        'dist_pico': dist_pico,
                        'longitud': l_carga
                    }                    
                
                if tipo_carga == "momento":

                    magnitud = float(input("Escriba la magnitud de la carga "))
                    distancia = float(input("Escriba la distancia de la carga al extremo izquierdo del tramo "))
                    sentido = input("Escriba si es sentidio horario o antihorario ")
                    
                    if sentido == "horario":
                        magnitud = -magnitud
                    
                    cargas['carga ' + str(i+1)]  = {
                        'tipo': tipo_carga,
                        'magnitud': magnitud,
                        'distancia': distancia,
                        'sentido': sentido
                    }

                
                if tipo_carga == "trapesoidal":
                    
                    magnitud_M = float(input("Escriba la magnitud del lado mayor "))
                    dist_M = float(input("Escriba la distancia del lado mayor al extremo izquierdo del tramo "))
                    
                    magnitud_m = float(input("Escriba la magnitud del lado menor "))
                    dist_m = float(input("Escriba la distancia del lado menor al extremo izquierdo del tramo "))
                    
                    l_carga = float(input("Escriba la longitud de la carga "))
                    
                    cargas['carga ' + str(i+1)]  = {
                        'tipo': tipo_carga,
                        'mag_mayor': magnitud_M,
                        'dist_mayor': dist_M,
                        'mag_menor': magnitud_m,
                        'dist_menor': dist_m,
                        'l_carga': l_carga
                    }
                
                print("----")
     
            L = float(input("Escriba la longitud del tramo "))
            
            tramo = {
                'voladizo': voladizo,
                'empotrado': empotrado,
                'cargas': cargas,
                'n_cargas': n_cargas,
                'L': L
            }
            
            self.tramos.append(tramo)
            
            mas_tramos = input("Hay mas tramos? ")
            
            print("----")
            print("----")
            print("\n")
            
            if mas_tramos == 'no':
                aux = False
        

In [4]:
Mi_viga = viga()

In [5]:
Mi_viga.tramos

[]

In [6]:
Mi_viga.definir_tramos()

Es un voladizo? si
Cuantas cargas hay en el tramo? 2
Escriba el tipo de la carga 1 puntual
Escriba la magnitud de la carga 1
Escriba la distancia de la carga al extremo izquierdo del tramo 0
----
Escriba el tipo de la carga 2 distribuida
Escriba la magnitud de la carga 2
Escriba la distancia de la carga al extremo izquierdo del tramo 0
Escriba la longitud de la carga 2
----
Escriba la longitud del tramo 2
Hay mas tramos? si
----
----


Es un voladizo? no
Es un empotramiento? no
Cuantas cargas hay en el tramo? 2
Escriba el tipo de la carga 1 puntual
Escriba la magnitud de la carga 3
Escriba la distancia de la carga al extremo izquierdo del tramo 3
----
Escriba el tipo de la carga 2 distribuida
Escriba la magnitud de la carga 2
Escriba la distancia de la carga al extremo izquierdo del tramo 3
Escriba la longitud de la carga 3
----
Escriba la longitud del tramo 6
Hay mas tramos? si
----
----


Es un voladizo? no
Es un empotramiento? no
Cuantas cargas hay en el tramo? 1
Escriba el tipo de 

In [7]:
Mi_viga.tramos

[{'voladizo': 'si',
  'empotrado': 'no',
  'cargas': {'carga 1': {'tipo': 'puntual', 'magnitud': 1.0, 'distancia': 0.0},
   'carga 2': {'tipo': 'distribuida',
    'magnitud': 2.0,
    'distancia': 0.0,
    'longitud': 2.0}},
  'n_cargas': 2,
  'L': 2.0},
 {'voladizo': 'no',
  'empotrado': 'no',
  'cargas': {'carga 1': {'tipo': 'puntual', 'magnitud': 3.0, 'distancia': 3.0},
   'carga 2': {'tipo': 'distribuida',
    'magnitud': 2.0,
    'distancia': 3.0,
    'longitud': 3.0}},
  'n_cargas': 2,
  'L': 6.0},
 {'voladizo': 'no',
  'empotrado': 'no',
  'cargas': {'carga 1': {'tipo': 'puntual',
    'magnitud': 4.0,
    'distancia': 2.0}},
  'n_cargas': 1,
  'L': 5.0},
 {'voladizo': 'no',
  'empotrado': 'no',
  'cargas': {'carga 1': {'tipo': 'distribuida',
    'magnitud': 2.0,
    'distancia': 0.0,
    'longitud': 5.0}},
  'n_cargas': 1,
  'L': 5.0}]

In [ ]:
tramos = len(Mi_viga.tramos)

t_ini = Mi_viga.tramos[0]
t_fin = Mi_viga.tramos[-1]

# Caso 1
if t_ini['voladizo'] == "si" and t_fin['voladizo'] == "si":
    indices = [x for x in range(1:len(tramos))]

    
# Caso 2    
if t_ini['voladizo'] == "si" and t_fin['voladizo'] == "no" and t_fin['empotrado'] == "no":

    
# Caso 3    
if t_ini['voladizo'] == "no" and t_ini['empotrado'] == "no" and t_fin['voladizo'] == "si":

    
# Caso 4    
if t_ini['voladizo'] == "no" and t_ini['empotrado'] == "no" and t_fin['voladizo'] == "no" and t_fin['empotrado'] == "no":

    
# Caso 5    
if t_ini['empotrado'] == "si" and t_fin['voladizo'] == "no" and t_fin['empotrado'] == "no":

    
# Caso 6    
if t_ini['empotrado'] == "si" and t_fin['empotrado'] == "si":

    
# Caso 7
if t_ini['empotrado'] == "si" and t_fin['voladizo'] == "si":

    
# Caso 8    
if t_ini['voladizo'] == "si" and t_fin['empotrado'] == "si":

    
# Caso 9    
if t_ini['voladizo'] == "no" and t_ini['empotrado'] == "no" and t_fin['empotrado'] == "si":

In [8]:
t_ini = Mi_viga.tramos[0]
t_fin = Mi_viga.tramos[-1]

In [12]:
calcular_voladizo(t_ini,0)

-6.0